 This notebook may not work and will be relaced with several proper notebooks explaining the whole process. 
 Getting FEs of four small supercells of Veliparib using OPLS forcefield:

In [ ]:

name = 'veliparib'
n_atoms_mol = 34
T = 300
FF_name = 'OPLS'
atom_order_PDB_match_itp = False
PME_cutoff = 0.35

form_cell_pairs = [['I','111'], ['I','121'], ['II','131'], ['II','221']]

''' equilibrate.py

from O.MM.sc_system import *
for form_cell in form_cell_pairs:
    form, cell = form_cell
    try:
        PDB_ideal_supercell = f'./O/MM/GAFF_sc/{name}/{name}_{form}_unitcell_cell{cell}.pdb'
        sc = SingleComponent(PDB = PDB_ideal_supercell, n_atoms_mol=n_atoms_mol, name=name, FF_name=FF_name, atom_order_PDB_match_itp=atom_order_PDB_match_itp)
        sc.initialise_system_(PME_cutoff=PME_cutoff, nonbondedMethod=app.PME)
        sc.initialise_simulation_(timestep_ps=0.002, P=1, T=T, barostat_type=2)
        sc.run_simulation_(50000, 50)
        sc._xyz = tidy_crystal_xyz_(sc.xyz, sc.boxes, n_atoms_mol=sc.n_atoms_mol, ind_rO=10)
        sc.save_simulation_data_(f'./O/MM/GAFF_sc/{name}/data/{name}_{FF_name.lower()}_NPT_dataset_Form_{form}_Cell_{cell}_Temp_{T}')
        index = get_index_average_box_automatic_(sc.boxes)
        sc.save_pdb_(sc.xyz[index], b=sc.boxes[index], name=f'./O/MM/GAFF_sc/{name}/{name}_{FF_name.lower()}_equilibrated_Form_{form}_Cell_{cell}_Temp_{T}.pdb')
        print('finished:',form,cell)
        del sc
    except:
        print('!! problem in:',form,cell)
        
'''

''' run_nvt.py

from O.MM.sc_system import *
for form_cell in form_cell_pairs:
    form, cell = form_cell
    try:
        PDB_eqm_supercell = f'./O/MM/GAFF_sc/{name}/{name}_{FF_name.lower()}_equilibrated_Form_{form}_Cell_{cell}_Temp_{T}.pdb'
        sc = SingleComponent(PDB = PDB_eqm_supercell, n_atoms_mol=n_atoms_mol, name=name, FF_name=FF_name, atom_order_PDB_match_itp=atom_order_PDB_match_itp)
        sc.initialise_system_(PME_cutoff=PME_cutoff, nonbondedMethod=app.PME)
        sc.initialise_simulation_(timestep_ps = 0.002, P = None, T = T, minimise = True)
        sc.simulation.step(200*50)
        sc.run_simulation_(sc.n_mol*18750, 50)
        sc.save_simulation_data_(f'./O/MM/GAFF_sc/{name}/data/{name}_{FF_name.lower()}_NVT_dataset_Form_{form}_Cell_{cell}_Temp_{T}')
        print('finished:',form,cell)
        del sc
    except: 
    
        print('!! problem in:',form,cell)
'''

''' train_pgm.py

from O.interface import *

for form_cell in form_cell_pairs:
    form, cell = form_cell
    if form == 'I': n_mol_unitcells = [8]
    else: n_mol_unitcells = [3]
    try:
        nn = NN_interface_sc_multimap(
                        name = f'{name}_{FF_name.lower()}_Form_{form}_Cell_{cell}_Temp_{T}',
                        paths_datasets = [f'./O/MM/GAFF_sc/{name}/data/{name}_{FF_name.lower()}_NVT_dataset_Form_{form}_Cell_{cell}_Temp_{T}',],
                        running_in_notebook = False,
                        training = True,
                        model_class = PGMcrys_v1,
                        )
        nn.set_ic_map_step1(ind_root_atom=5, option=0)
        nn.set_ic_map_step2(check_PES=True)
        nn.set_ic_map_step3(n_mol_unitcells=n_mol_unitcells)
        nn.set_model(n_layers = 4, learning_rate=0.001, n_att_heads=4, evaluation_batch_size=5000)
        nn.set_trainer(n_batches_between_evaluations=50)
        n_batches = 5000
        print('training for n_batches =',n_batches)
        nn.train(n_batches = n_batches, save_misc = True, save_BAR = True, test_inverse = False, evaluate_on_training_data = False)
        nn.load_misc_()
        nn.solve_BAR_using_pymbar_(rerun=True)
        nn.save_samples_(20000)
        nn.save_model_()
        print('finished:',form,cell)
        del nn
    except: 
        print('!! problem in:',form,cell)
        
'''

''' run_ecm.py (optional)

form_cell_pairs = [['I','111'],['II','131'],]

from O.MM.ecm_basic import *

def veliparib_ARGS_oss_OPLS(form, cell):
    args_initialise_object = {
        'PDB' : f'./O/MM/GAFF_sc/{name}/{name}_opls_equilibrated_Form_{form}_Cell_{cell}_Temp_{T}.pdb',
        'n_atoms_mol': 34,
        'name': 'veliparib',
        'FF_name' : 'OPLS',
        'atom_order_PDB_match_itp' : atom_order_PDB_match_itp,
    }
    args_initialise_system = {
        'PME_cutoff': 0.35,
        'removeCMMotion': True,
        'nonbondedMethod': app.PME,
        'custom_EwaldErrorTolerance': 0.0001,
        'constraints': None,   
    }
    args_initialise_simulation = {
        'rbv': None,
        'minimise': True,
        'T': 300,
        'timestep_ps': 0.0005, # !
        'collision_rate': 1,
        'P': None, # NVT
        'barostat_type': 1,
        'barostat_1_scaling': [True, True, True],
        'stride_barostat': 25,
    }
    ARGS_oss = [args_initialise_object, args_initialise_system, args_initialise_simulation]
    return ARGS_oss

for form_cell in form_cell_pairs:
    form, cell = form_cell
    try:
        t0 = time.time()
        ecm = ECM_basic(
                        name = f'ecm_{name}_{FF_name.lower()}_Form_{form}_Cell_{cell}_Temp_{T}_FA5_',
                        working_dir_folder_name = './O/MM/GAFF_sc/ecm_runs/',
                        ARGS_oss = veliparib_ARGS_oss_OPLS(form=form, cell=cell),
                        k_EC = 6000.0,
                        COM_removal_by_fixing_one_atom_index_of_this_atom = 5,
                        overwrite = False,
                        path_lambda_1_dataset = f'./O/MM/GAFF_sc/{name}/data/{name}_{FF_name.lower()}_NVT_dataset_Form_{form}_Cell_{cell}_Temp_{T}',
                        )
        ecm.unsupervised_FE_(
                batch_size_increments = 20000,
                max_dataset_size_per_lambda = 60000,
                max_n_lambdas = 30,
                SE_tol_per_molecule = 0.046,
                )
                
        ecm.estimate_FE_using_mBAR_()
        t1 = time.time()
        print('finished:',form,cell)
        time_min = (t1-t0)/60.
        print(form, cell, 'took', time_min, 'minutes') # egrep -H "minutes" nohup.out
        del ecm
    except: 
        print('!! problem in:',form,cell)

'''

In [ ]:
''' other notes about ECM_basic :
# can ignore message "not converged within temperature tolerance", can check that temperatures were converged later.

# not uncommon: if MBAR doesn't converge: 
    1) can decrease SE_tol_per_molecule and/or increase max_n_lambdas
    2) after this change to the script was made, run the .py script again (with overwrite = False) 

# ergodicity: does not show up in error bars. lambda states close to 1 (weakly perturbed) need to be sampled longer!
    # TODO: add a method that samples those states well, not treating them like all the other perturbed states.

# rare: if any simulation breaks due to timestep, or diverges (but does not completely break), this will show up in:
    # log_of_BAR_results : BAR estimates as a function of 'ecm progress' curve
    # cumulative average energy during each simulation, this is also in log_of_BAR_results (raw energies in lambda_evaluations)
    # any such lambda dataset needs to be carefully removed from lambda_evaluations, xyz, and others files related to it.
    # It is sometimes easier to just start again from scratch if this happens. TODO: add an automated method to patch breakages.

TODO: add method for a better definition of new lambda-states compared to 'half_way'. Use energy overlaps instead.
TODO: tidy up ECM_basic.
'''